In [1]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,8 hours 11 mins
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,"7 days, 12 hours and 18 minutes"
H2O_cluster_name:,H2O_from_python_kevin_0sjooy
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,654 Mb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [2]:
train = pd.read_csv('input/train.csv', index_col=0)
pred = pd.read_csv('input/predict.csv', index_col=0)


In [3]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train["cut_le"] = le.fit_transform(train["cut"])
train["color_le"] = le.fit_transform(train["color"])
train["clarity_le"] = le.fit_transform(train["clarity"])

In [4]:
train = train[['carat','depth','table','x','y','z','price','cut_le','color_le','clarity_le']]

In [5]:
from scipy import stats
import numpy as np
z = np.abs(stats.zscore(train))
print(z)
threshold = 3
print(np.where(z > 3))
train = train[(z < 3).all(axis=1)]

[[0.67782057 0.1011905  0.69550817 ... 0.43631756 0.8271011  1.06219822]
 [0.71993701 1.21676264 1.55080885 ... 1.41173902 0.9377797  0.482915  ]
 [0.2145398  1.00759287 2.49256178 ... 1.41173902 1.5260733  0.482915  ]
 ...
 [0.2145398  2.33233478 1.10154544 ... 1.51452537 0.2388075  0.482915  ]
 [0.86734453 1.29327468 0.65228204 ... 1.51452537 0.8271011  1.06219822]
 [0.81731285 0.24063702 0.65228204 ... 0.53910391 0.8271011  1.25493464]]
(array([   78,   116,   134, ..., 40422, 40438, 40447]), array([6, 6, 1, ..., 6, 6, 2]))


In [6]:
h2train = h2o.H2OFrame(train)
h2pred = h2o.H2OFrame(pred)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [7]:
x = list(train.drop(columns = 'price').columns)
y = 'price'

In [9]:
automl = H2OAutoML(max_models = 50, seed = 1, max_runtime_secs = 1200, sort_metric = 'RMSE')
automl.train(x,y,training_frame=h2train)

AutoML progress: |███████████████████████Failed polling AutoML progress log: Unexpected HTTP error: ('Connection aborted.', BadStatusLine('GET /3/Jobs/$03017f00000132d4ffffffff$_800af0b211c33505ac5f19a45b634d97 HTTP/1.1\r\n'))
█████████████████████████████████| 100%


In [10]:
leader_board = automl.leaderboard
leader_board.head()

model_id,rmse,mean_residual_deviance,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20201006_034914,439.333,193013,193013,235.996,0.0923109
StackedEnsemble_BestOfFamily_AutoML_20201006_034914,440.929,194418,194418,237.212,0.0927938
GBM_2_AutoML_20201006_034914,445.302,198293,198293,243.171,0.0975692
GBM_3_AutoML_20201006_034914,445.71,198658,198658,240.285,0.0944433
GBM_4_AutoML_20201006_034914,449.713,202241,202241,238.361,0.0888556
GBM_1_AutoML_20201006_034914,454.545,206611,206611,250.809,0.102799
XGBoost_grid__1_AutoML_20201006_034914_model_3,459.552,211188,211188,244.791,0.0928794
XGBoost_grid__1_AutoML_20201006_034914_model_1,461.88,213333,213333,243.85,0.0915032
XGBoost_grid__1_AutoML_20201006_034914_model_2,463.432,214769,214769,245.284,0.0911571
XGBoost_3_AutoML_20201006_034914,472.403,223165,223165,261.609,0.107358


In [11]:
predictions = automl.leader.predict(h2pred)
predictions.head()

stackedensemble prediction progress: |████████████████████████████████████| 100%


/usr/local/lib/python3.8/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'cut_le': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.8/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'color_le': substituting in a column of NaN
  warnings.warn(w)
/usr/local/lib/python3.8/site-packages/h2o/job.py:70: UserWarning: Test/Validation dataset is missing column 'clarity_le': substituting in a column of NaN
  warnings.warn(w)


predict
712.033
6371.27
5148.21
818.2
2500.2
613.796
1592.96
687.738
9067.28
942.106


In [14]:
prd = predictions.as_data_frame()
columns=['id','price']
prd = prd.reset_index(level=0)
prd.columns = columns
prd.to_csv('lt.csv', index = False)